In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
# read comments cvs
df_comments = pd.read_csv("Manual checking - comments.csv")

In [3]:
# df_comments

In [4]:
# read answers cvs
df_answers = pd.read_csv("Manual checking - answers.csv")

In [5]:
# df_answers

In [6]:
def split_true_false(df):
    df_true = df[df['outdated (manually checked)'] == True]
    df_false = df[df['outdated (manually checked)'] == False]
    return df_true,df_false

In [7]:
def tags_count(tgs):
    service = []
    for items in tgs:
        if pd.isna(items)==False:
            line = list(items.split(","))
            service.extend(line)
    return service

In [8]:
def tags_to_dict(df_tags, tag_list):
    tag_dict = {}
    for item in df_tags:
        tag_dict[item] = 0
    for i in tag_list:
        tag_dict[i]+=1
    return tag_dict

In [9]:
def get_df_keys(df_tags):
    key_= Counter(df_tags).keys()
    value_=Counter(df_tags).values()
    dict_df= dict(zip(key_,value_))
    return dict_df

In [10]:
df_tags = pd.read_csv("service_counts_6000_samples.csv")

In [11]:
# pd.DataFrame(list(tags_dict_sorted.items()), columns = ['service','frequency'])

In [12]:
def get_dict(df):
    df_true,df_false = split_true_false(df)
    tags_df =tags_count(df['related service'])
    tags_true =tags_count(df_true['related service'])
    dict_df= get_df_keys(tags_df)
    dict_true = tags_to_dict(tags_df,tags_true)
    return dict_df,dict_true

In [13]:
dict_df_comments,dict_true_comments =get_dict(df_comments)
df_comments_tags = pd.DataFrame(list(dict_df_comments.items()), columns = ['service','comment_frequency'])
df_comments_true_tags = pd.DataFrame(list(dict_true_comments.items()), columns = ['service','true_comment_frequency'])
df_tags = df_tags.merge(df_comments_tags,how = 'left',on = 'service')
df_tags = df_tags.merge(df_comments_true_tags,how = 'left',on = 'service')

In [14]:
df_tags 

,service,frequency,comment_frequency,true_comment_frequency
0,amazon-s3,1057,6.0,6.0
1,amazon-ec2,972,4.0,1.0
2,aws-lambda,687,3.0,1.0
3,amazon-web-services,590,NaN,NaN
4,amazon-dynamodb,382,1.0,1.0
...,...,...,...,...
222,amazon-workdocs,1,NaN,NaN
223,aws-shell,1,NaN,NaN
224,amazon-kinesis-video-streams,1,NaN,NaN
225,amazonads,1,NaN,NaN


In [15]:
dict_df_answers,dict_true_answers =get_dict(df_answers)
df_answers_tags = pd.DataFrame(list(dict_df_answers.items()), columns = ['service','answers_frequency'])
df_answers_true_tags = pd.DataFrame(list(dict_true_answers.items()), columns = ['service','true_answers_frequency'])
df_tags = df_tags.merge(df_answers_tags,how = 'left',on = 'service')
df_tags = df_tags.merge(df_answers_true_tags,how = 'left',on = 'service')

In [16]:
# df_tags.to_csv("tags_table.csv")

In [17]:
df_tags= df_tags.fillna(0)

In [18]:
df_tags['sample_frequency'] = df_tags['comment_frequency']+df_tags['answers_frequency']
df_tags['sample_true_frequency'] = df_tags['true_comment_frequency']+df_tags['true_answers_frequency']

In [19]:
df_tags = df_tags[['service', 'frequency','sample_frequency',
       'sample_true_frequency']]

tags_compare = df_tags[df_tags['sample_frequency']+df_tags['sample_true_frequency']!=0]

In [20]:
tags_compare.to_csv("tags_distribution.csv")

In [21]:
tags_compare

,service,frequency,sample_frequency,sample_true_frequency
0,amazon-s3,1057,7.0,6.0
1,amazon-ec2,972,8.0,2.0
2,aws-lambda,687,3.0,1.0
4,amazon-dynamodb,382,1.0,1.0
5,amazon-elastic-beanstalk,306,1.0,1.0
6,aws-sdk,203,2.0,0.0
13,aws-cli,115,1.0,0.0
14,amazon-ecs,111,2.0,2.0
18,aws-glue,83,1.0,0.0
20,amazon-emr,80,2.0,2.0
